In [1]:
import os
import sys
sys.path.append('..')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings.local")

import django
django.setup()

In [2]:
import ciprs_reader

In [3]:
from dear_petition.petition.models import CIPRSRecord

In [4]:
record = CIPRSRecord.objects.all()[0]

In [5]:
record

<CIPRSRecord: CIPRSRecord object (2)>

In [6]:
record.report_pdf.name

'ciprs/cypress-example_RmdMP0h.PDF'

In [7]:
from django.core.files.storage import get_storage_class

In [8]:
storage = get_storage_class()()

In [9]:
storage.location

'/app/dear_petition/media'

In [10]:
path = os.path.join(storage.location, record.report_pdf.name)
path

'/app/dear_petition/media/ciprs/cypress-example_RmdMP0h.PDF'

In [13]:

reader.json()

'{\n    "General": {\n        "County": "ORANGE",\n        "File No": "15IF702186"\n    },\n    "Case Information": {\n        "Case Status": "DISPOSED",\n        "Offense Date": "2015-05-17T21:59:00"\n    },\n    "Defendant": {\n        "Date of Birth/Estimated Age": "1988-09-10",\n        "Name": "BOWES,DANIEL,PATRICK",\n        "Race": "WHITE",\n        "Sex": "MALE"\n    },\n    "Offense Record": {\n        "Records": [\n            {\n                "Action": "CHARGED",\n                "Description": "SPEEDING(80 mph in a 65 mph zone)",\n                "Severity": "INFRACTION",\n                "Law": "G.S. 20-141(B)",\n                "Code": "4450"\n            },\n            {\n                "Action": "ARRAIGNED",\n                "Description": "SPEEDING(80 mph in a 65 mph zone)",\n                "Severity": "INFRACTION",\n                "Law": "G.S. 20-141(B)",\n                "Code": "4450"\n            },\n            {\n                "Action": "CONVICTED",\n    

In [20]:
import tempfile
import json
from django.core.files.storage import FileSystemStorage
import subprocess
import logging

In [23]:
logger = logging.getLogger('test')

with tempfile.TemporaryDirectory(prefix='ciprs-') as tmp_folder:
    storage = FileSystemStorage(location=tmp_folder)
    storage.save('record.pdf', record.report_pdf)
    saved_file_path = os.path.join(storage.location, 'record.pdf')
    reader = ciprs_reader.PDFToTextReader(saved_file_path)
    try:
        reader.parse()
        record.data = json.loads(reader.json())
    except subprocess.CalledProcessError as e:
        logger.exception(e)
        record.data = {'error': str(e)}
    finally:
        record.save()

In [22]:
record.data

{'error': "Command 'pdftotext -layout -enc UTF-8 /tmp/ciprs-jkdul6kk/record.pdfa -' returned non-zero exit status 1."}

In [53]:
record.data = None
record.save()

In [8]:
json.loads(record.data)

{'General': {'County': 'ORANGE', 'File No': '15IF702186'},
 'Case Information': {'Case Status': 'DISPOSED',
  'Offense Date': '2015-05-17T21:59:00'},
 'Defendant': {'Date of Birth/Estimated Age': '1988-09-10',
  'Name': 'BOWES,DANIEL,PATRICK',
  'Race': 'WHITE',
  'Sex': 'MALE'},
 'Offense Record': {'Records': [{'Action': 'CHARGED',
    'Description': 'SPEEDING(80 mph in a 65 mph zone)',
    'Severity': 'INFRACTION',
    'Law': 'G.S. 20-141(B)',
    'Code': '4450'},
   {'Action': 'ARRAIGNED',
    'Description': 'SPEEDING(80 mph in a 65 mph zone)',
    'Severity': 'INFRACTION',
    'Law': 'G.S. 20-141(B)',
    'Code': '4450'},
   {'Action': 'CONVICTED',
    'Description': 'IMPROPER EQUIP - SPEEDOMETER',
    'Severity': 'INFRACTION',
    'Law': 'G.S. 20-123.2',
    'Code': '4418'}],
  'Disposed On': '2015-07-15',
  'Disposition Method': 'DISPOSED BY JUDGE'}}

In [43]:
storage.location

'/tmp/ciprs-mhezuxip'

In [44]:
storage.save('ciprs.pdf', record.report_pdf)

'ciprs.pdf'

In [3]:
from django.urls import reverse

In [5]:
reverse('admin:petition_ciprsrecord_changelist')

'/admin/petition/ciprsrecord/'